In [1]:
import numpy as np
import xarray as xr
from tqdm import tqdm
from xcape.core import calc_cape
import os

In [2]:
idx=100
ds_t = xr.open_dataset(
        "/scratch/nf33/hk25_LSP/Germany/MC/ta_6hourly.nc").isel(time=idx)

ds_tdew = xr.open_dataset(
    "/scratch/nf33/hk25_LSP/Germany/MC/tdew_6hourly.nc").isel(time=idx)



In [3]:
## flip the pressure levels bcos so it goes from surface (descending order)
ta_da = ds_t["ta"].stack(ngrid=('latitude','longitude'))\
.isel(pressure=slice(None, None, -1)).transpose('ngrid','pressure')-273.15
tdew_da = ds_tdew["tdew"].stack(ngrid=('latitude','longitude'))\
.isel(pressure=slice(None, None, -1)).transpose('ngrid','pressure')-273.15

ta_surf = ds_t["ta"].isel(pressure=-1).stack(ngrid=('latitude','longitude')).values.flatten()-273.15
tdew_surf = ds_tdew["tdew"].isel(pressure=-1).stack(ngrid=('latitude','longitude')).values.flatten()-273.15
        


In [9]:
ta_val = ta_da.load().values
td_val = tdew_da.load().values
psurf = np.zeros(ta_val.shape[0])
psurf[:] = 1000
# ICON pressure is in Pa not hPa
plvl = ds_t.pressure.isel(pressure=slice(None, None, -1)).values.astype(float)/100        

cape, cin, mulev, zmulev = calc_cape(plvl, ta_val, td_val, 
                                         psurf,ta_surf, 
                                         tdew_surf,
                                         source ='most-unstable', vertical_lev="pressure")

In [10]:
cape

array([1697.5957  , 1675.0566  , 1538.769   , ...,  504.79498 ,
        171.46666 ,   38.352276], dtype=float32)